In [14]:
import os

In [9]:
%pwd

'/Users/kjaggav1/Library/CloudStorage/OneDrive-JCPenney/myworkspace/practice/foodclassification'

In [11]:
%pwd

'/Users/kjaggav1/Library/CloudStorage/OneDrive-JCPenney/myworkspace/practice'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [17]:

from src.foodclassification.constants import *
from src.foodclassification.utils.common import read_yaml, create_directories


ModuleNotFoundError: No module named 'box'

In [8]:
!brew install pip


==> Downloading https://formulae.brew.sh/api/formula.jws.json

==> Downloading https://formulae.brew.sh/api/cask.jws.json
######################################################################### 100.0%
pip is part of the python formula:
  brew install python


In [2]:
%pwd

'/Users/kjaggav1/Library/CloudStorage/OneDrive-JCPenney/myworkspace/practice/foodclassification/research'

In [3]:
os.chdir("../")

NameError: name 'os' is not defined